<a href="https://colab.research.google.com/github/yilmazde/Python-Assignment/blob/master/brainAGE_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Predicting the BrainAGE of Schizophrenia patients**

# The steps of a simple machine learning (ML) process to predict age from brain data 

### 1) Train a model on predicting the age (= outcome variable, in ML terminology referred to as **target**) of participants by using structural MRI (sMRI) data of healthy participants (predictor variables = **features**): 
###     cortical thickness & subcortical volume = training data

### 2) The output will be a model that best fits the training data

### 3) By using this model, we can now predict the age of scizophrenia patients from their sMRI data. 



# In this tutorial, we will work on the first step of training to find the best fitting model. Training is a fancy way of saying "parameter estimation" in ML lingo.

## Let's delete all output first: Edit -> select all cells -> R-click -> Clear selected outputs

## A) INSTALLING & IMPORTING PACKAGES

In [7]:
# Typically run from the terminal to install the necessary packages 

!pip install numpy scipy scikit-learn pandas joblib pytorch
!pip install deap update_checker tqdm stopit xgboost
!pip install tpot

# Importing necessary packages to our working environment

import sklearn
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tpot import TPOTRegressor

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached pytorch-1.0.2.tar.gz (689 bytes)
  ERROR: Failed building wheel for pytorch
  Running setup.py clean for pytorch
Failed to build pytorch
    Running setup.py install for pytorch ... error
ERROR: Command errored out with exit status 1: /usr/bin/python3 -u -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'/tmp/pip-install-prhlaf9v/pytorch_5f8f26f7273346ff88e9f6abaad8518e/setup.py'"'"'; __file__='"'"'/tmp/pip-install-prhlaf9v/pytorch_5f8f26f7273346ff88e9f6abaad8518e/setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' install --record /tmp/pip-record-k7ys0gdj/install-record.txt --single-version-externally-managed --compile --install-headers /usr/

## B) UPLOAD & PREPARE OUR DATA

In [8]:
# upload the dataset (hcp_to_csv.csv) to the environment -> R-click &copy path 

hcp_data = pd.read_csv('/content/HCP_to_tpot.csv')


In [9]:
hcp_data.head(n=10)


,subject_no,education,gender,age,BMI,FS_L_LatVent_Vol,FS_L_InfLatVent_Vol,FS_L_Cerebellum_WM_Vol,FS_L_Cerebellum_Cort_Vol,FS_L_ThalamusProper_Vol,...,FS_R_Rostralanteriorcingulate_Thck,FS_R_Rostralmiddlefrontal_Thck,FS_R_Superiorfrontal_Thck,FS_R_Superiorparietal_Thck,FS_R_Superiortemporal_Thck,FS_R_Supramarginal_Thck,FS_R_Frontalpole_Thck,FS_R_Temporalpole_Thck,FS_R_Transversetemporal_Thck,FS_R_Insula_Thck
0,100004,14.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,100206,16.0,M,27.0,26.64,5057.0,399.0,16808.0,62713.0,10426.0,...,3.115,2.515,2.731,2.278,3.032,2.660,2.641,3.579,3.147,3.278
2,100307,16.0,F,27.0,22.96,3559.0,103.0,13146.0,61980.0,7787.0,...,3.012,2.789,3.087,2.250,3.151,2.766,3.675,4.026,2.819,3.002
3,100408,16.0,M,33.0,27.75,3910.0,51.0,17022.0,66666.0,9097.0,...,2.862,2.502,2.760,2.282,2.846,2.554,2.840,3.102,2.532,2.947
4,100610,16.0,M,27.0,36.91,4262.0,266.0,15463.0,66293.0,9613.0,...,3.336,2.620,3.013,2.257,2.901,2.632,2.994,3.723,2.721,3.068
5,101006,12.0,F,35.0,30.27,4321.0,874.0,15105.0,52591.0,7838.0,...,2.807,2.581,2.864,2.313,2.879,2.674,2.658,3.540,2.726,2.958
6,101107,12.0,M,22.0,21.13,4733.0,254.0,13699.0,55760.0,8452.0,...,3.278,2.727,3.038,2.502,3.151,2.895,2.976,3.956,2.649,3.346
7,101208,17.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,101309,16.0,M,29.0,22.20,2364.0,327.0,14657.0,57649.0,9394.0,...,3.206,2.638,2.890,2.467,2.945,2.667,2.778,3.408,2.694,3.009
9,101410,16.0,M,29.0,24.27,7146.0,95.0,15645.0,63474.0,8365.0,...,2.943,2.745,2.842,2.304,2.990,2.647,3.001,3.493,2.750,3.033


In [10]:
hcp_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1206 entries, 0 to 1205
Columns: 117 entries, subject_no to FS_R_Insula_Thck
dtypes: float64(115), int64(1), object(1)
memory usage: 1.1+ MB


In [11]:
hcp_data.describe()

,subject_no,education,age,BMI,FS_L_LatVent_Vol,FS_L_InfLatVent_Vol,FS_L_Cerebellum_WM_Vol,FS_L_Cerebellum_Cort_Vol,FS_L_ThalamusProper_Vol,FS_L_Caudate_Vol,...,FS_R_Rostralanteriorcingulate_Thck,FS_R_Rostralmiddlefrontal_Thck,FS_R_Superiorfrontal_Thck,FS_R_Superiorparietal_Thck,FS_R_Superiortemporal_Thck,FS_R_Supramarginal_Thck,FS_R_Frontalpole_Thck,FS_R_Temporalpole_Thck,FS_R_Transversetemporal_Thck,FS_R_Insula_Thck
count,1206.000000,1204.000000,1112.000000,1111.000000,1112.000000,1112.000000,1112.000000,1112.000000,1112.000000,1112.000000,...,1112.000000,1112.000000,1112.000000,1112.000000,1112.000000,1112.000000,1112.000000,1112.000000,1112.000000,1112.000000
mean,374551.585406,14.863787,28.801259,26.536436,6477.440647,219.692446,14576.747302,57055.011691,8458.993705,3809.406475,...,3.014303,2.587173,2.872945,2.324103,2.943936,2.700318,2.860307,3.646102,2.735424,3.012008
std,272686.898230,1.819279,3.698478,5.197908,3614.207150,133.897084,1924.235484,6106.332438,917.416068,476.316625,...,0.200786,0.115119,0.126878,0.099872,0.126345,0.111477,0.203171,0.300554,0.166674,0.140783
min,100004.000000,11.000000,22.000000,16.480000,1512.000000,7.000000,9536.000000,40122.000000,5945.000000,2575.000000,...,2.368000,2.115000,2.414000,1.930000,2.486000,2.297000,1.899000,2.434000,2.183000,2.505000
25%,154254.250000,13.750000,26.000000,22.865000,4047.750000,127.000000,13255.500000,52604.500000,7785.000000,3479.750000,...,2.883750,2.510750,2.786000,2.256750,2.861000,2.626750,2.723000,3.468750,2.626000,2.928000
50%,212166.500000,16.000000,29.000000,25.530000,5578.500000,192.000000,14462.000000,56839.000000,8417.500000,3777.500000,...,3.010000,2.583000,2.871000,2.321000,2.941500,2.700000,2.860000,3.676500,2.736000,3.013500
75%,586310.500000,16.000000,32.000000,29.370000,7986.750000,276.000000,15806.500000,61361.500000,9109.500000,4102.750000,...,3.148500,2.664250,2.957250,2.391000,3.032000,2.777000,2.994250,3.855250,2.848250,3.107000
max,996782.000000,17.000000,37.000000,47.760000,25798.000000,1040.000000,27358.000000,75365.000000,11399.000000,5546.000000,...,3.743000,2.961000,3.292000,2.622000,3.336000,3.024000,3.675000,4.455000,3.389000,3.435000


#### Gender is a letter-coded categorical variable, but our model cannot work with letters. Let us recode it in numbers!

In [12]:
hcp_data['gender'].value_counts()

F    605
M    507
Name: gender, dtype: int64

In [13]:
# turn males to 0 females to 1
hcp_data.gender[hcp_data.gender == 'M'] = 0  
hcp_data.gender[hcp_data.gender == 'F'] = 1

<ipython-input-13-0213f54ecae0>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hcp_data.gender[hcp_data.gender == 'M'] = 0
<ipython-input-13-0213f54ecae0>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hcp_data.gender[hcp_data.gender == 'F'] = 1


#### Shuffling the data is preferred to avoid any preexisting ordering in the data

In [14]:
hcp_shuffle = hcp_data.iloc[np.random.permutation(len(hcp_data))]
hcp = hcp_shuffle.reset_index(drop=True) # resets the index numbers
hcp.head() # hcp is te new data name

,subject_no,education,gender,age,BMI,FS_L_LatVent_Vol,FS_L_InfLatVent_Vol,FS_L_Cerebellum_WM_Vol,FS_L_Cerebellum_Cort_Vol,FS_L_ThalamusProper_Vol,...,FS_R_Rostralanteriorcingulate_Thck,FS_R_Rostralmiddlefrontal_Thck,FS_R_Superiorfrontal_Thck,FS_R_Superiorparietal_Thck,FS_R_Superiortemporal_Thck,FS_R_Supramarginal_Thck,FS_R_Frontalpole_Thck,FS_R_Temporalpole_Thck,FS_R_Transversetemporal_Thck,FS_R_Insula_Thck
0,181131,12.0,1,33.0,29.09,5115.0,102.0,10145.0,44484.0,7761.0,...,2.937,2.339,2.643,2.289,2.882,2.577,2.562,3.654,2.497,2.986
1,632845,12.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,239944,16.0,1,32.0,25.21,8577.0,54.0,13359.0,56365.0,8522.0,...,3.074,2.553,2.829,2.339,2.983,2.719,2.620,3.578,2.824,2.518
3,149337,16.0,0,33.0,21.48,7735.0,327.0,15102.0,61522.0,7989.0,...,3.299,2.415,2.654,2.099,2.848,2.468,2.660,3.442,2.566,2.808
4,152225,17.0,0,35.0,26.87,8735.0,239.0,13953.0,58605.0,9415.0,...,3.038,2.425,2.574,2.256,2.909,2.550,2.601,3.584,2.690,3.103


In [15]:
pd.isnull(hcp).any()

subject_no                      False
education                        True
gender                           True
age                              True
BMI                              True
                                ...  
FS_R_Supramarginal_Thck          True
FS_R_Frontalpole_Thck            True
FS_R_Temporalpole_Thck           True
FS_R_Transversetemporal_Thck     True
FS_R_Insula_Thck                 True
Length: 117, dtype: bool

In [16]:
hcp = hcp.dropna()
pd.isnull(hcp).any()

subject_no                      False
education                       False
gender                          False
age                             False
BMI                             False
                                ...  
FS_R_Supramarginal_Thck         False
FS_R_Frontalpole_Thck           False
FS_R_Temporalpole_Thck          False
FS_R_Transversetemporal_Thck    False
FS_R_Insula_Thck                False
Length: 117, dtype: bool

## C) Define Target & Features

### Store the variable to be predicted later = age = TARGET variable

In [17]:
age_target = hcp['age'].values  

### Store the predictor variables to be used for age prediction later = edu, bmi, gender, thickness, vol = FEATURES 

In [18]:
hcp_features = hcp.drop('age', axis=1)


## D) Train-Test Split 

https://www.youtube.com/watch?v=rCevxk3jeKs&t=45s




In [19]:
X_train, X_test, y_train, y_test = train_test_split(hcp_features, age_target,
                                                    train_size=0.75, test_size=0.25, 
                                                    random_state=42)

## E) Linear Regression 

### "The TPOTRegressor performs an intelligent search over machine learning pipelines that can contain supervised regression models, preprocessors, feature selection techniques, and any other estimator or transformer that follows the scikit-learn API." (http://epistasislab.github.io/tpot/api/)

### Note: allow GPU to speed up the process from: Edit -> Notebook Settings

In [20]:
tpot = TPOTRegressor(verbosity=2, cv = 10) 

### TPOT is so smart it does the cross-validation all by itself !
### Cross-validation refers to using different parts of the data as training & testing sets to ensure an aggregate unbiased result that is representative of all data. If you would like to follow-up on cross-validation: https://www.youtube.com/watch?v=fSytzGwwBVw&t=5s

### tpot.fit starts the paramter estimation and optimizes pipelines
### It will optimize forever (...up to several days...), the longer it runs, the better the fit.

In [21]:

tpot.fit(X_train, y_train)
print(tpot.score(X_test, y_test))
tpot.export('tpot_brainage_pipeline.py')

Optimization Progress:   0%|          | 0/10100 [00:00<?, ?pipeline/s]



TPOT closed during evaluation in one generation.


TPOT closed prematurely. Will use the current best pipeline.

Best pipeline: RidgeCV(input_matrix)
-11.829908551634194


## F) The End

### The output scrip looks like something like this

In [ ]:
# Average CV score on the training set was: -50.96662428137334
exported_pipeline = make_pipeline(
    StackingEstimator(estimator=RandomForestRegressor(bootstrap=True, max_features=0.55, min_samples_leaf=20, min_samples_split=12, n_estimators=100)),
    StackingEstimator(estimator=LassoLarsCV(normalize=True)),
    MaxAbsScaler(),
    StackingEstimator(estimator=GradientBoostingRegressor(alpha=0.75, learning_rate=0.1, loss="lad", max_depth=2, max_features=0.7500000000000001, min_samples_leaf=2, min_samples_split=11, n_estimators=100, subsample=0.35000000000000003)),
    SelectFwe(score_func=f_regression, alpha=0.003),
    MaxAbsScaler(),
    StackingEstimator(estimator=SGDRegressor(alpha=0.01, eta0=0.1, fit_intercept=True, l1_ratio=0.0, learning_rate="invscaling", loss="huber", penalty="elasticnet", power_t=0.5)),
    SelectFwe(score_func=f_regression, alpha=0.016),
    ExtraTreesRegressor(bootstrap=True, max_features=0.8, min_samples_leaf=4, min_samples_split=7, n_estimators=100)
)

# you can predict age now for a new dataset

predicted_age_excv0 = exported_pipeline.predict(features)
metrics.r2_score(tpot_data['age'], predicted_age_excv0)

# calculate brainAGE

brainAGE_cdp = predicted_age_cdp - y_test